In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable
from torch import Tensor, optim, nn
import wandb
from tqdm import tqdm
import pprint

wandb.login()

wandb: Currently logged in as: moritz-palm. Use `wandb login --relogin` to force relogin


True

In [2]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.02, 0.05, 0.1, 0.2]
        },
        'epochs': {
            'values': [50, 100, 200]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'num_layers': {
            'values': [1, 2, 3, 4]
        },
        'hidden_size': {
            'values': [32, 64, 128]
        },
        'dropout_prob': {
            'values': [0.0, 0.1, 0.2, 0.3]
        },
        'regularization_lambda': {
            'values': [0.0, 0.01, 0.1, 1.0]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'loss': {
            'values': ['CrossEntropyLoss']
        },
        'activation': {
            'values': ['ReLU']
        },
        'input_size': {
            'value': 229
        }
    }
}

In [3]:
pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'activation': {'values': ['ReLU']},
                'batch_size': {'values': [32, 64, 128]},
                'dropout_prob': {'values': [0.0, 0.1, 0.2, 0.3]},
                'epochs': {'values': [50, 100, 200]},
                'hidden_size': {'values': [32, 64, 128]},
                'input_size': {'value': 229},
                'learning_rate': {'values': [0.01, 0.02, 0.05, 0.1, 0.2]},
                'loss': {'values': ['CrossEntropyLoss']},
                'num_layers': {'values': [1, 2, 3, 4]},
                'optimizer': {'values': ['adam', 'sgd']},
                'regularization_lambda': {'values': [0.0, 0.01, 0.1, 1.0]}}}


In [4]:
sweep_id = wandb.sweep(sweep_config, project='leaguify')

Create sweep with ID: 8g5ov2yg
Sweep URL: https://wandb.ai/moritz-palm/leaguify/sweeps/8g5ov2yg


In [5]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver**********
CUDNN version: 8801

Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [6]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

In [7]:
def get_train_data(slice=1):
    full_dataset = StaticDataset('../data/processed/train_static.npy')
    sub_dataset = torch.utils.data.Subset(full_dataset, range(0, len(full_dataset), slice))
    train_data = torch.utils.data.Subset(sub_dataset, range(0, int(len(sub_dataset) * 0.8)))
    val_data = torch.utils.data.Subset(sub_dataset, range(int(len(sub_dataset) * 0.8), len(sub_dataset)))
    return train_data, val_data

In [8]:
def get_test_data():
    return StaticDataset('../data/processed/test_static.npy')

In [9]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, num_classes=2):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        for i in range(num_layers):
            if i == 0:
                self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
            else:
                self.linear_relu_stack.append(nn.Linear(hidden_size, hidden_size))
            self.linear_relu_stack.append(nn.ReLU())
            self.linear_relu_stack.append(self.dropout)
        self.linear_relu_stack.append(nn.Linear(hidden_size, num_classes))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    else:
        raise ValueError("Optimizer not supported")
    return optimizer

In [12]:
def train_batch(matches, labels, model, optimizer, criterion):
    output = model(matches)
    loss = criterion(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [13]:
def train_log(loss, example_count, epoch):
    wandb.log({"epoch": epoch, "loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [14]:
def train(config=None):
    with wandb.init(config=config):

        config = wandb.config  #
        train_data, val_data = get_train_data(slice=1)
        train_loader = make_loader(train_data, batch_size=config.batch_size)
        val_loader = make_loader(val_data, batch_size=config.batch_size)
        model = NeuralNetwork(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob).to(device)
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        print(f'optimizer: {optimizer}')
        criterion = nn.CrossEntropyLoss()
        wandb.watch(model, criterion, log="all", log_freq=10)

        total_batches = len(train_loader) * config.epochs
        example_count = 0
        batch_count = 0
        for epoch in tqdm(range(config.epochs)):
            for _, (matches, labels) in enumerate(train_loader):
                loss = train_batch(matches, labels, model, optimizer, criterion)
                example_count += len(matches)
                batch_count += 1
                if (batch_count + 1) % 25 == 0:
                    train_log(loss, example_count, epoch)
        test(model, val_loader)

In [15]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"val_accuracy": correct / total})

In [ ]:
wandb.agent(sweep_id, train, count=50)

wandb: Agent Starting Run: 0t00s7ad with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	dropout_prob: 0
wandb: 	epochs: 50
wandb: 	hidden_size: 32
wandb: 	input_size: 229
wandb: 	learning_rate: 0.01
wandb: 	loss: CrossEntropyLoss
wandb: 	num_layers: 1
wandb: 	optimizer: adam
wandb: 	regularization_lambda: 0


optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


  2%|▏         | 1/50 [00:02<01:48,  2.21s/it]

Loss after 00768 examples: 0.550
Loss after 01568 examples: 0.655
Loss after 02368 examples: 0.541
Loss after 03158 examples: 0.474


  4%|▍         | 2/50 [00:02<00:49,  1.03s/it]

...

Loss after 129898 examples: 0.313
Loss after 130688 examples: 0.380
Loss after 131488 examples: 0.376


 98%|█████████▊| 49/50 [00:12<00:00,  4.24it/s]

Loss after 132288 examples: 0.370
Loss after 133078 examples: 0.407
Loss after 133878 examples: 0.438


100%|██████████| 50/50 [00:12<00:00,  3.88it/s]

Loss after 134678 examples: 0.321
Loss after 135478 examples: 0.407
Accuracy of the model on the 678 test matches: 70.353982%
